# **run load_data.ipynb BEFORE running this!**

In [17]:
import pickle
import os.path
import numpy as np
normalized_data_file_name = 'data_normalized'
with open(normalized_data_file_name + '.pickle', 'rb') as handle:
    spectrum, temperature = pickle.load(handle)
    print(f"read data from {normalized_data_file_name}.pickle")
print(f"shape of spectrum data: {spectrum.shape}")
print(f"shape of temperature data: {temperature.shape}")
print()
print(f"there are {temperature.shape[0]} spectrums")
print(f"each spectrum is {spectrum.shape[1]} long")

read data from data_normalized.pickle
shape of spectrum data: (6000, 10000)
shape of temperature data: (6000, 1)

there are 6000 spectrums
each spectrum is 10000 long


In [18]:
indices_file_name = 'cross_validation_resample=2_fold=5'
with open(indices_file_name + '.pickle', 'rb') as handle:
    train_indices, test_indices = pickle.load(handle)
    print(f"got indices from {indices_file_name}.pickle")  
print()
print(f"sets of training indices: {len(train_indices)}")
print(f"number of training indices per set: {len(train_indices[0])}")
print(f"sets of testing indices: {len(test_indices)}")
print(f"number of testing indices per set: {len(test_indices[0])}")

got indices from cross_validation_resample=2_fold=5.pickle

sets of training indices: 10
number of training indices per set: 4800
sets of testing indices: 10
number of testing indices per set: 1200


In [19]:
input_dimension = spectrum.shape[1]
print(f"input dimension is: {input_dimension}")

input dimension is: 10000


In [20]:
import torch
import torch.nn as nn
class Model(torch.nn.Module):
    def __init__(self, device, input_dim=input_dimension):
        super().__init__()
        self.relu  = nn.ReLU()
        self.hidden_dim = 500
        self.linear1 = torch.nn.Linear(input_dim, self.hidden_dim)
        self.linear2 = torch.nn.Linear(self.hidden_dim, self.hidden_dim)
        self.linear3 = torch.nn.Linear(self.hidden_dim, 1)
        self.device = device
        self.to(device)
    def forward(self, x):
        y = self.linear3(self.relu(self.linear2(self.relu(self.linear1(x)))))
        # change: remove sigmoid, use y result as is
        # y = torch.sigmoid(y)
        return y

In [21]:
from torch.utils.data import DataLoader
from torch import optim
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class CalculateMSE():
    def __init__(self, net, n_epochs, batch_size):
        super().__init__()
        self.net = net
        #initialize some constants
        self.batch_size = 32
        self.learning_rate = 1e-4
        self.n_epochs = n_epochs
        self.net.apply(self.weights_init)   
    def weights_init(self,layer):
        if type(layer) == nn.Linear:
            nn.init.orthogonal_(layer.weight)
    def get_mse(self,train_data, train_label, test_data, test_label):
        train_set = torch.utils.data.TensorDataset(
            torch.Tensor(train_data).to(device), 
            torch.Tensor(train_label).to(device))
        val_set = torch.utils.data.TensorDataset(
            torch.Tensor(test_data).to(device), 
            torch.Tensor(test_label).to(device))
        loader_args = dict(batch_size=self.batch_size)
        train_loader = DataLoader(train_set, shuffle=True, drop_last=True, **loader_args)
        val_loader = DataLoader(val_set, shuffle=True, drop_last=True, **loader_args)
        tloss = []
        vloss = []
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.net.parameters(), lr=self.learning_rate) # weight_decay=0
        for epoch in range(0, self.n_epochs):
            # if epoch % 1000 == 0:
            #     print(f"epoch = {epoch}")
            epoch_train_loss=[]
            for i, data in enumerate(train_loader, 0):
                inputs, label = data
                y_pred = self.net(inputs.to(self.net.device))
                loss = criterion(y_pred, label.to(self.net.device))
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                epoch_train_loss.append(loss.item())
            tloss.append(np.mean(epoch_train_loss))
            epoch_loss=[]
            for i, data in enumerate(val_loader, 0):
                with torch.no_grad():
                    inputs1, label1 = data
                    y_pred1 = self.net(inputs1.to(self.net.device))
                    loss1 = criterion(y_pred1, label1.to(self.net.device))
                    epoch_loss.append(loss1.item())
            vloss.append(np.mean(epoch_loss))
        return np.min(vloss), self.net


In [22]:
from pathlib import Path
from datetime import datetime
n_epochs=1000
batch_size=32

PATH = 'model_dnn/'
Path(PATH).mkdir(parents=True, exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# change: commented out: alraedy read in X, y data earlier
# response = pd.read_csv("1127_final_data/response.csv", header=None).values #input X
# spectra = pd.read_csv("1127_final_data/spectra.csv", header=None).values #ground truth label Y
# change: input dim = 1000
mdl = Model(device=device, input_dim=input_dimension)
losses = []
print(f"number of epochs: {n_epochs}, batch size: {batch_size}, device: {mdl.device}")

f = open("log_dnn.txt", "w")
f.write("train START: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
f.write(f"number of epochs: {n_epochs}, batch size: {batch_size}, device: {device}\n")
for i,(train,test) in enumerate(zip(train_indices,test_indices)):
    print(f"we are on fold no.{i}")
    train_data, train_label= spectrum[train],temperature[train]
    test_data, test_label= spectrum[test],temperature[test]
    mse_calculator = CalculateMSE(mdl,n_epochs,batch_size)
    # print(f"train_data shape: {train_data.shape}")
    # print(f"train_label shape: {train_label.shape}")
    loss,model = mse_calculator.get_mse(train_data, 
                                        train_label, 
                                        test_data, 
                                        test_label)
    losses.append(loss)
    print(f"\tloss: {loss}")
    print("\ttime: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        
    f.write(f"fold = {i}")
    f.write(f"\tloss: {loss}\n")
    f.write("\ttime: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") +"\n")

    torch.save(model.state_dict(), PATH+'model_'+str(i))
f.write("train END: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
f.close()

number of epochs: 1, batch size: 32, device: cuda
we are on fold no.0
	loss: 0.029884868112669605
	time: 2023-05-19 19:36:29
we are on fold no.1
	loss: 0.0605871417635196
	time: 2023-05-19 19:36:31
we are on fold no.2
	loss: 0.017533550089275516
	time: 2023-05-19 19:36:32
we are on fold no.3
	loss: 0.025228089904664335
	time: 2023-05-19 19:36:33
we are on fold no.4
	loss: 0.01526148579511288
	time: 2023-05-19 19:36:34
we are on fold no.5
	loss: 0.03049465862883104
	time: 2023-05-19 19:36:35
we are on fold no.6
	loss: 0.0335914027449247
	time: 2023-05-19 19:36:36
we are on fold no.7
	loss: 0.03412169328815228
	time: 2023-05-19 19:36:37
we are on fold no.8
	loss: 0.01194183060244934
	time: 2023-05-19 19:36:38
we are on fold no.9
	loss: 0.01568076746991357
	time: 2023-05-19 19:36:39


In [23]:
print(f"mean losses: {np.mean(losses)}, std: {np.std(losses)}")

mean losses: 0.027432548839951287, std: 0.013514232723174664


In [24]:
number_figures = 10
import matplotlib.pyplot as plt

indices = torch.randint(0,len(spectrum),(number_figures,)).unique()
for i in indices:
    print(f"we use {i}th example")
    # change: cast i to int, since pandas not work with torch.int64
    # changed: removed figure, since the output is just one number
    spec = np.asarray(spectrum[int(i)]).flatten()
    temp = np.asarray(temperature[int(i)]).flatten()
    # plt.figure(i)

    prediction = model(torch.Tensor(np.asarray(spectrum[int(i)])).to(model.device)).detach().cpu().flatten()
    # plt.plot(prediction)
    print(f"\tthe prediction (normalized) is: {prediction.item()}")
    # plt.plot(temp)
    # plt.legend(['reconstruction','ground truth'])
    print(f"\tthe ground truth (normalized) is: {temp.item()}")


we use 65th example
	the prediction (normalized) is: -1.4382696151733398
	the ground truth (normalized) is: -1.5849058664490416
we use 1159th example
	the prediction (normalized) is: -0.6137189269065857
	the ground truth (normalized) is: -0.8138705800684255
we use 1609th example
	the prediction (normalized) is: -0.8813304901123047
	the ground truth (normalized) is: -0.8138705800684255
we use 3332th example
	the prediction (normalized) is: 0.5775017142295837
	the ground truth (normalized) is: 0.4711882305659315
we use 4479th example
	the prediction (normalized) is: 1.068681240081787
	the ground truth (normalized) is: 0.9852117548196725
we use 4880th example
	the prediction (normalized) is: 1.1047797203063965
	the ground truth (normalized) is: 0.9852117548196725
we use 4930th example
	the prediction (normalized) is: 1.0836238861083984
	the ground truth (normalized) is: 0.9852117548196725
we use 5298th example
	the prediction (normalized) is: 1.1263225078582764
	the ground truth (normaliz

# **skip cell 14 for now (it's in template file!)**